In [1]:
import pandas as pd
import numpy as np
import requests as req
import json

In [2]:
aave_url = "https://api.thegraph.com/subgraphs/name/aave/protocol-v2"

# User Transaction Data
Obtain data for every transaction every user has done. Types of transactions are:
- **deposit:** User deposits funds to use as collateral.


- **redeem:** User redeems aTokens (obtained when depositing collateral) for their underlying asset. Effectively doing a withdraw from their collateral. 


- **borrow:** User borrows assets against the deposited collateral or against credit delegated by another user.


- **repay:** User repays previous borrowing of an asseet


- **swap:** User changes the borrow mode either from stable to variable or from variable to stable.


- **rebalance:** Stable borrow rates are updated according to the new pool data.


- **liquidation:** User's position is liquidated when their loan's health factor gets below the liquidation threshold, losing the deposited colateral.

In [8]:
users_query = """
query userBorrowHistory($lastUserId: String = "0x000000000000512644392395185ad535410f98c2") {
  users(first: 1000, orderBy: id, where:{
    id_gt:$lastUserId
  }) {
    id
    depositHistory(first: 1000, orderBy: timestamp){
      ...TransactionFields
      amount
      reserve {
        ...ReserveFields
      }
    }
    redeemUnderlyingHistory(first: 1000, orderBy: timestamp){
      ...TransactionFields
      amount
      reserve {
        ...ReserveFields
      }
    }
    borrowHistory(first: 1000, orderBy: timestamp){
      ...TransactionFields
      amount
      stableTokenDebt
      variableTokenDebt
      borrowRateMode
      borrowRate
      reserve {
        ...ReserveFields
      }
    }
    repayHistory(first: 1000, orderBy: timestamp){
      ...TransactionFields
      amount
      reserve {
        ...ReserveFields
      }
    }
    swapHistory(first: 1000, orderBy: timestamp){
      ...TransactionFields
      borrowRateModeFrom
      borrowRateModeTo
      stableBorrowRate
      variableBorrowRate
      reserve{
        ...ReserveFields
      }
    }
    rebalanceStableBorrowRateHistory(first: 1000, orderBy: timestamp){
      ...TransactionFields
      borrowRateFrom
      borrowRateTo
      reserve{
        ...ReserveFields
      }
    }
    liquidationCallHistory(first: 1000, orderBy: timestamp){
      ...TransactionFields
      collateralAmount
      principalAmount
      collateralReserve {
        ...ReserveFields
      }
      principalReserve {
        ...ReserveFields
      }
    }
  }
}

fragment TransactionFields on UserTransaction{
  id
  timestamp
}

fragment ReserveFields on Reserve{
  id
  symbol
  decimals
}
"""

In [9]:
users_payload = {
    'query' : users_query,
    'variables': {
        'lastUserId': "0x0000000000000000000000000000000000000001"
    }
}

In [10]:
userData = []

remainingUsers = []

while True:
    request = req.post(aave_url, json = users_payload)
    result = json.loads(request.content)
    filtered_result =[user for user in result['data']['users'] if not (
            len(user['depositHistory']) == 1000 or
            len(user['redeemUnderlyingHistory']) == 1000 or
            len(user['borrowHistory']) == 1000 or
            len(user['repayHistory']) == 1000 or
            len(user['rebalanceStableBorrowRateHistory']) == 1000 or
            len(user['swapHistory']) == 1000)]
            
        
    userData.extend(filtered_result)
    users_payload['variables']['lastUserId'] = userData[-1]['id']
    print(len(userData),userData[-1]['id'])
    if len(result['data']['users']) < 1000:
        break
    

1000 0x026544dacfafc3422a0219cb6be03ecb5d99a771
2000 0x04fb5857bd5d98780602b84bf03eabdb0d3ff90b
3000 0x07be1d15a904d7a033e31cfd517a14bb553afa1e
4000 0x0a5cc302d6e6185192db8d5e375e7b683c4a3952
5000 0x0d2e0083dde4d115f842c6cfc59ed17cca6bb7c9
6000 0x0fc73d5df9256ec376f31d82f5460e48f314b918
7000 0x126c1322de76b74b64de3ec85344b4d251ff13f9
7999 0x1519e84999d6dbde35b00f2e73b18aa83b27290a
8999 0x17ae0a4dc5f3a17f00fcd366a3e1c96a01c2e4f5
9999 0x1a30d14c23750aa54bc7e4854a3a9ffb5da3ffd7
10999 0x1cd919e5625a2604107f20f60ace89a3f7f52916
11999 0x1f847fd5e08fb559a69280a14e7e904e6dbff81f
12997 0x2210e6b99e28d4eba28740b7b9835a252620a49d
13997 0x24cf8b2e3673600d77b3d4b08769fad321ad0354
14996 0x276b9dae2b199c437b2abd43746bea503b55234f
15996 0x2a0f26c4e457fad576cb5b01bfca4d120ddb1ceb
16996 0x2cb568b64b4c82176d8a28d785c416009eda9c21
17996 0x2f686f1cf743bd2d274a61dd12ca6eb8af65c745
18996 0x3201f3b96bc535f4dc9cdafeec9aaca1ca65fe97
19996 0x34abea0ab4601f91aa8054e1810879d9441954be
20996 0x37584fe0d49e0b9d7295b1

In [11]:
with open('aave_user_transaction_data_complete.json', 'w') as datafile:
    json.dump(userData, datafile)

# Reserve Data 
Obtain data for every asset reserve, specifically historical data about the Loan-To-Value ratio and liquidation threshold.  

In [12]:
reserve_query = """
query ReserveHistory{
  reserves(first: 60){
    id
    symbol
    decimals
    lastUpdateTimestamp
    baseLTVasCollateral
    reserveLiquidationThreshold
    configurationHistory(orderBy:timestamp){
      id
      timestamp
      baseLTVasCollateral
      reserveLiquidationThreshold
    }
  }
}
"""

In [13]:
reserve_payload = {
    'query' : reserve_query,
}

In [14]:
request = req.post(aave_url, json = reserve_payload)
result = json.loads(request.content)

In [15]:
reserve_data = result['data']['reserves']

In [16]:
for reserve in result['data']['reserves']:
    reserve['configurationHistory'].append(
        {
            'id':'actual',
            'timestamp':  2**32,
            'baseLTVasCollateral': reserve['baseLTVasCollateral'],
            'reserveLiquidationThreshold': reserve['reserveLiquidationThreshold']
        }
    )

In [17]:
with open('aave_reserve_configuration_data_complete.json', 'w') as datafile:
    json.dump(reserve_data, datafile)

In [23]:
simple_reserve_query = """
query Reserves{
  reserves(first: 60){
    id
    underlyingAsset
    symbol
    decimals
    lastUpdateTimestamp
    baseLTVasCollateral
    reserveLiquidationThreshold
  }
}
"""

In [24]:
simple_reserve_payload = {
    'query' : simple_reserve_query,
}

In [25]:
request = req.post(aave_url, json = simple_reserve_payload)
result = json.loads(request.content)

In [26]:
reserve_data = result['data']['reserves']

In [27]:
reserve_data

[{'id': '0x0000000000085d4780b73119b644ae5ecd22b3760xb53c1a33016b2dc2ff3653530bff1848a515c8c5',
  'underlyingAsset': '0x0000000000085d4780b73119b644ae5ecd22b376',
  'symbol': 'TUSD',
  'decimals': 18,
  'lastUpdateTimestamp': 1663308059,
  'baseLTVasCollateral': '8000',
  'reserveLiquidationThreshold': '8250'},
 {'id': '0x004375dff511095cc5a197a54140a24efef3a4160xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5',
  'underlyingAsset': '0x004375dff511095cc5a197a54140a24efef3a416',
  'symbol': 'AmmUniWBTCUSDC',
  'decimals': 18,
  'lastUpdateTimestamp': 1651982413,
  'baseLTVasCollateral': '6000',
  'reserveLiquidationThreshold': '7000'},
 {'id': '0x03ab458634910aad20ef5f1c8ee96f1d6ac549190xb53c1a33016b2dc2ff3653530bff1848a515c8c5',
  'underlyingAsset': '0x03ab458634910aad20ef5f1c8ee96f1d6ac54919',
  'symbol': 'RAI',
  'decimals': 18,
  'lastUpdateTimestamp': 1663287251,
  'baseLTVasCollateral': '0',
  'reserveLiquidationThreshold': '0'},
 {'id': '0x056fd409e1d7a124bd7017459dfea2f387b6d5cd0xb53c1

In [22]:
with open('aave_reserve_data_simple.json', 'w') as datafile:
    json.dump(reserve_data, datafile)